In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

from astropy.io import fits
from astropy import table as t

import gpflow as gpf

In [2]:
from gpflow.utilities import print_summary
from gpflow.ci_utils import ci_niter

FLOAT_TYPE = np.float64
gpf.config.set_default_float(FLOAT_TYPE)
gpf.config.set_default_summary_fmt("notebook")
np.random.seed(0)

MAXITER = ci_niter(10000)

In [3]:
boss_dupl = fits.open('cov/boss_duplicate_spectra.fit')
manga_dupl = fits.open('cov/manga_duplicate_spectra.fit')

In [4]:
manga_dupl_tab = t.Table.read(manga_dupl)

spec1, spec2 = np.array(manga_dupl_tab['SPEC1']), np.array(manga_dupl_tab['SPEC2'])
err1, err2 = np.array(manga_dupl_tab['ERR1']), np.array(manga_dupl_tab['ERR2'])
wave = np.array(manga_dupl_tab['WAVE'][0])
wave_log = np.log10(wave)

In [5]:
boss_dupl_tab = t.Table.read(boss_dupl)

In [12]:
spec1, spec2 = np.array(boss_dupl_tab['SPEC1']), np.array(boss_dupl_tab['SPEC2'])
med = np.median(np.stack([spec1, spec2]), axis=(0, 2))[:, None]
keepspec = med.flatten() > 0
spec1, spec2 = spec1 / med, spec2 / med
err1, err2 = np.array(boss_dupl_tab['ERR1']) / med, np.array(boss_dupl_tab['ERR2']) / med

spec1, spec2, err1, err2 = spec1[keepspec], spec2[keepspec], err1[keepspec], err2[keepspec]

/home/zpace/miniconda3/envs/gpflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/zpace/miniconda3/envs/gpflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [13]:
wave = np.array(boss_dupl_tab['WAVE'][0])
wave_log = np.log10(wave)

In [14]:
def coregion_coords(Xs, n_output=None):
    """generate coordinate matrix for coregion GP
    """
    # start out evaluating dimensionality
    
    if n_output is not None:
        if type(Xs) is list:
            assert len(Xs) == n_output, 'for pre-specified Xs, length of list must match n_output'
        else:
            Xs = [Xs for _ in range(n_output)]
            
    Xs_with_coord = np.row_stack([np.column_stack([Xs[i], np.full(Xs[i].shape[0], i)]) for i in range(n_output)])
    
    return Xs_with_coord
            
def coregion_outputs(ys, n_output=None):
    if n_output is not None:
        if type(ys) is list:
            assert len(ys) == n_output, 'for pre-specified Xs, length of list must match n_output'
        else:
            ys = [ys for _ in range(n_output)]
            
    return np.concatenate(ys)

In [52]:
def optimize_model_with_scipy(model, data):
    optimizer = gpf.optimizers.Scipy()
    optimizer.minimize(
        model.training_loss_closure(data),
        variables=model.trainable_variables,
        method="l-bfgs-b",
        options={"disp": True, "maxiter": MAXITER})

def plot_model(m, Y, lower=-8.0, upper=8.0):
    plt.figure(figsize=(6, 4), dpi=200)
    pX = np.linspace(lower, upper, 100)[:, None]
    pY, pYv = m.predict_y(pX)
    if pY.ndim == 3:
        pY = pY[:, 0, :]
    plt.plot(X, Y, "x")
    plt.gca().set_prop_cycle(None)
    plt.plot(pX, pY)
    for i in range(pY.shape[1]):
        top = pY[:, i] + 2.0 * pYv[:, i] ** 0.5
        bot = pY[:, i] - 2.0 * pYv[:, i] ** 0.5
        plt.fill_between(pX[:, 0], top, bot, alpha=0.3)
    plt.xlabel("X")
    plt.ylabel("f")
    plt.title(f"ELBO: {m.elbo(data):.3}")
    plt.plot(Z, Z * 0.0, "o")
    
def plot_model_separate(m, Y, lower=-8.0, upper=8.0):
    npts, ngp = Y.shape
    
    ncols = 2
    nrows = ngp // ncols + 1 * ((ngp % ncols) > 0)
    cs, rs = 1.25, 1.
    bs = 1.
    fwid, fht = cs * ncols + 2. * bs, rs * nrows + 2. * bs
    
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(fwid, fht), dpi=200)
    
    pX = np.linspace(lower, upper, 100)[:, None]
    pY, pYv = m.predict_y(pX)
    if pY.ndim == 3:
        pY = pY[:, 0, :]
    
    for i, ax in enumerate(axs.flat):
        ax.plot(X, Y[:, i], linewidth=0.25)
        ax.plot(pX[:, 0], pY[:, i])
        
        top = pY[:, i] + 2.0 * pYv[:, i] ** 0.5
        bot = pY[:, i] - 2.0 * pYv[:, i] ** 0.5
        ax.fill_between(pX[:, 0], top, bot, alpha=0.3)
        
        ax.axhline(0., linestyle='--', c='k', zorder=0)
    
    plt.suptitle(f"ELBO: {m.elbo(data):.3}")

In [65]:
dspec = (spec1 - spec2)[:100, :].astype(FLOAT_TYPE)
err = np.sqrt(err1**2. + err2**2.)[:100, :].astype(FLOAT_TYPE)

In [66]:
N = dspec.shape[1]  # number of points
D = 1  # number of input dimensions
M = 20  # number of inducing points
L = 2  # number of latent GPs
P = dspec.shape[0]  # number of observations = output dimensions

print('\n'.join(['{}: {}'.format(par, val) for par, val in zip(['N', 'D', 'M', 'L', 'P'], [N, D, M, L, P])]))

N: 4563
D: 1
M: 20
L: 2
P: 100


In [67]:
X = wave_log[:, None].astype(FLOAT_TYPE)
data = X, dspec.T
Zinit = np.random.uniform(X.min(), X.max(), M)[:, None]

In [68]:
print(*[a.shape for a in data])

(4563, 1) (4563, 100)


In [69]:
# Create list of kernels for each output
kern_list = [gpf.kernels.SquaredExponential() + gpf.kernels.Polynomial() + gpf.kernels.Constant() for _ in range(P)]
# Create multi-output kernel from kernel list
kernel = gpf.kernels.SeparateIndependent(kern_list)
# initialization of inducing input locations (M random points from the training inputs)
Z = Zinit.copy()
# create multi-output inducing variables from Z
iv = gpf.inducing_variables.SharedIndependentInducingVariables(
    gpf.inducing_variables.InducingPoints(Z)
)

In [70]:
# create SVGP model as usual and optimize
m = gpf.models.SVGP(kernel, gpf.likelihoods.Gaussian(), inducing_variable=iv, num_latent_gps=P)
print_summary(m)

name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[0].kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[0].kernels[1].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[0].kernels[1].offset,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[0].kernels[2].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].kernels[0].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].kernels[1].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].kernels[1].offset,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].kernels[2].variance,Parameter,Softplus,,True,(),float64,1.0


In [71]:
optimize_model_with_scipy(m, data)

KeyboardInterrupt: 

In [ ]:
plot_model(m, dspec.T, X.min(), X.max())

In [ ]:
plot_model_separate(m, dspec.T, X.min(), X.max())

In [ ]:
print_summary(m)